# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*

## Практикум 3. Формат JSON и его обработка в рамках блока кода JavaScript

Импортируем все необходимое:

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Словари и датафреймы

Если нам нужно хранить много информации разного уровня в Python, это удобно делать с помощью словарей (тип `dict`):

In [ ]:
info = {"database" : "HP", 
 "data" : [{"name" : "Dobby", "type" : "house-elf"}, 
           {"name": "Harry Potter", "type": "human"}, 
           {"name" : "Griphook", "type" : "goblin"}]}
info

В таком же виде удобно хранить информацию где-то на сервере, чтобы в нужный момент выбирать по ключу подходящие данные и подставлять в нужное место на страницу сайта. Так, в примере выше, известно, что есть база данных `HP`, в ней есть набор данных `data`, в котором есть характеристики разных героев из вселенной Дж.Роулинг. 

Как преобразовать данные выше в более привычный табличный формат? Вспомнить, что функция `DataFrame()` из `pandas` умеет преобразовывать списки словарей в датафреймы (один словарь внутри списка равен одной строке таблицы). Давайте выберем список с характеристиками героев и получим удобную таблицу!

In [ ]:
pd.DataFrame(info["data"])

Теперь давайте разберемся, в каком виде могут храниться подобного вида данные в исходном коде веб-страницы.

### Формат JSON

JSON (от *JavaScript Object Notation*) – текстовый формат хранения данных, изначально использовался в языке JavaScript, но затем стал универсальным машиночитаемым форматом, распознаваемым разными языками программирования. Различают:

* JSON-строки (текст с определённой структурой данных внутри);
* JSON-файлы (текстовые файлы с расширением `.json` со строкой JSON-внутри).

Какие структуры данных Python могут встретиться внутри JSON-строки? Знакомые нам списки и словари!

**Пример JSON-строки, содержащей списки:**

In [ ]:
# фрагмент результатов голосования в Арбитражный комитет Википедии: 
# время голосования, голос, кандидат, избиратель:

example01 = """
[["2008-11-23 00:32:00", "-", "Solon", "Kalan"], 
  ["2008-11-23 00:34:00", "+", "Chronicler", "Altes"], 
  ["2008-11-23 00:34:00", "+", "Ilya Voyager", "Altes"]]
"""

**Пример JSON-строки, содержащей словари:**

In [ ]:
# фрагмент результатов голосования за актеров 
# на сайте kinoteatr.ru

example02 = """
[{ "id":"16804", "plus":"131", "minus":"4", "voted":"" },
{ "id":"56008", "plus":"91", "minus":"10", "voted":"" },
{ "id":"62460", "plus":"94", "minus":"4", "voted":"" }]
"""

Этот формат хранения данных удобен своей универсальностью. Во-первых, он позволяет сохранять и выгружать в компактные текстовые файлы данные со сложной структурой (например, словари, внутри которых есть ещё словари). Во-вторых, формат JSON не привязан к какому-то конкретному языку программирования. Можно создать список списков в Python, выгрузить его в строку JSON, затем считать эту строку с помощью другого языка и получить результат в виде аналогичных структур данных, принятых в этом языке (например, аналогом питоновского словаря `dict` в языке R может выступить поименованный вектор или фрейм `list`).

По этим причинам формат JSON очень популярен. Его можно встретить при работе с географическими данными (файлы с особым расширением `.geojson`, которые содержат метки с координатами объектов), при парсинге HTML-страниц (файлы `.json`, из которых «подтягивается» регулярно обновляемая информация для построения всяких интерактивных визуализаций на сайте) и при подключении к API – интерфейсам, которые можно использовать как базы данных для автоматизированной выгрузки данных из приложений и социальных сетей.

В этом практикуме мы поработаем с JSON-строками. Импортируем необходимый для работы модуль `json`. Этот модуль `json` – базовый (как знакомые нам `requests` или `os`), он не требует дополнительной установки.  

In [ ]:
import json

В модуле `json` есть две функции, `loads()` и `load()`. Первая преобразует данные из обычной строки (как здесь), вторая – данные, загруженные из файла с расширением `.json`. Преобразование JSON-строки в структуру данных называется **десериализацией JSON**. 

Обратная операция – превращение структуры данных в Python в JSON-строку – тоже существует, и называется она **сериализацией JSON**. Для сериализации используется аналогичная пара функций, `dumps()` и `dump()`. Первая будет превращать структуру данных в JSON-строку, вторая – превращать структуру данных в строку и выгружать эту строку в файл с расширением `.json`.

В качестве наглядного примера десериализуем строки `example01` и `example02` – превратим валидные (корректные, где все скобки и кавычки на месте) JSON-сроки в питоновские списки и словари:

In [ ]:
# строка с текстом -> список списков
json.loads(example01)

In [ ]:
# строка с текстом -> список словарей
json.loads(example02)

И сериализуем словарь `info`:

In [ ]:
# словарь -> текст
json.dumps(info)

In [ ]:
# словарь -> файл .json
f = open("info.json", "w")
json.dump(info, f)
f.close()

### Извлечение кода JavaScript из HTML и обработка JSON-строк

Вернемся к задаче сбора ссылок на новости по дате. В предыдущем практикуме мы написали код, который находит нужный фрагмент с кодом на JavaScript (с тэгом `<script>`), в котором явно есть ссылки на новости:

In [ ]:
url = "https://nplus1.ru/news/2025/01/03"
soup = BeautifulSoup(requests.get(url).text)
js = soup.find_all("script")[8].text

Объект `js` сейчас – это обычная строка (текст) с кодом на языке JavaScript. Знать этот язык нам совсем не обязательно, достаточно понять, какой фрагмент текста содержит ссылки. В данном случае текст в `js` выглядит жутко, это из-за того, что двойные кавычки местами представлены в виде символов Unicode. Выполним замену, чтобы меньше пугаться:

In [ ]:
js = js.replace("\\u0022", '"')

Чтобы еще меньше пугаться, давайте заменим блоки из `\` на один слэш:

In [ ]:
js_clean = js.replace("\\\\\\", "")

В целом, уже сейчас можно попытаться найти в `js_clean` фрагменты со ссылками на новости, но это неудобно. Поэтому давайте выберем текст после `JSON.parse(` и попробуем привести его к «чистой» JSON-строке!

In [ ]:
# разбиваем по JSON.parse(\' и забираем все, что после (индекс 1)
# метод .strip() без текста внутри убирает отступы в начале/конце строки,
# а с текстом внутри – убирает этот текст в начале/конце строки

final = js_clean.split("JSON.parse(\'")[1].strip().strip("\');")
# final

Теперь эту строку можно десериализовать – превратить в словарь!

In [ ]:
D = json.loads(final)

Давайте выберем из словаря запись с нужными данными и превратим их в таблицу!

In [ ]:
### YOUR CODE HERE ###

### Задача 1

Выберите из полученного датафрейма столбец со ссылками и превратите его в массив или список ссылок (тип `list` или `array`).

In [ ]:
### YOUR CODE HERE ###

### Задача 2

Напишите функцию `get_url_by_date()`, которая принимает на вход ссылку на страницу новостей за определенную дату и возвращает список ссылок на все новости в этот день.

In [ ]:
### YOUR CODE HERE ###

Теперь мы можем совместить код из практикума 2.2 для получения ссылок на страницы всех дат (задача №2 в сюжете 2) и код выше, чтобы получить программу, которая сформирует список ссылок на все новости за 2024 год.

Однако мы можем пойти дальше и получить гораздо больше информации из датафрейма, который получился преобразованием JSON-строки с кода внутри `<script>` на странице. Давайте в свободном режиме изучим структуру этого датафрейма!

In [ ]:
# полезный код для перекодирования текста

"\\u0422\\u0435".encode().decode('unicode_escape')